In [1]:
# recommended to use virtual env to install pip package
# see link official docs for installation reference: https://github.com/googleapis/python-analytics-data#installation
%pip install google-analytics-data

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
import datetime
import pandas as pd
import numpy as np

In [3]:
from google.analytics.data_v1beta import BetaAnalyticsDataClient
from google.analytics.data_v1beta.types import (
    DateRange,
    Dimension,
    Metric,
    RunReportRequest,
    OrderBy,
)

In [4]:
# modified from official google docs for GA4: 
# https://developers.google.com/analytics/devguides/reporting/data/v1/quickstart-client-libraries

def pull_from_ga_into_df(dimensions, metrics, order_by):
    """Runs a simple report on a Google Analytics 4 property."""
    # TODO(developer): Uncomment this variable and replace with your
    #  Google Analytics 4 property ID before running the sample.
    # property_id = "YOUR-GA4-PROPERTY-ID"

    # Using a default constructor instructs the client to use the credentials
    # specified in GOOGLE_APPLICATION_CREDENTIALS environment variable.
    client = BetaAnalyticsDataClient()
    property_id="353368209"

    request = RunReportRequest(
        property=f"properties/{property_id}",
        dimensions=[Dimension(name=d) for d in dimensions],
        metrics=[Metric(name=m) for m in metrics],
        date_ranges=[DateRange(start_date="30daysAgo", end_date="today")],
        order_bys=[OrderBy(desc=True, dimension=OrderBy.DimensionOrderBy(dimension_name=order_by))]
    )
    response = client.run_report(request)
    
    ## process into pd dataframe
    
    # get headers
    data_dict = {}
    for d in response.dimension_headers:
        data_dict[d.name] = []

    for m in response.metric_headers:
        data_dict[m.name] = []

    # insert into dictionary
    for r in response.rows:
        vals = [v.value for v in [*r.dimension_values, *r.metric_values]]
        for i, key in enumerate(data_dict):
            data_dict[key].append(vals[i])

    df = pd.DataFrame(data=data_dict)
    
    return df

In [5]:
query_map = {}

In [6]:
query_map["AudienceOverview"] = {
    "metrics": [
        "averageSessionDuration",
        "bounceRate",
        "newUsers",
        "sessionsPerUser",
        "screenPageViewsPerSession",
        "screenPageViews",
        "sessions",
        "activeUsers", # not sure if this is what we want for the api
    ],
    "dimensions": [
        "date",
        "hostName"
    ],
    "order_by": "date"
}

In [7]:
audience_overview_df = pull_from_ga_into_df(
    query_map["AudienceOverview"]["dimensions"], 
    query_map["AudienceOverview"]["metrics"],
    query_map["AudienceOverview"]["order_by"],
)
audience_overview_df

,date,hostName,averageSessionDuration,bounceRate,newUsers,sessionsPerUser,screenPageViewsPerSession,screenPageViews,sessions,activeUsers
0,20230310,moandbear.com.ph,7.0485653333333333,1,2,1.5,1.3333333333333333,4,3,2
1,20230309,moandbear.com.ph,46.534410384615384,0.69230769230769229,10,1.0833333333333333,1.4615384615384615,19,13,12
2,20230308,moandbear.com.ph,90.478372153846152,0.30769230769230771,10,1.0833333333333333,2.3846153846153846,31,13,12
3,20230307,moandbear.com.ph,356.8390975,0.33333333333333331,6,1,2.3333333333333335,14,6,6
4,20230306,connectsys.shop,507.05419616666671,0.5,6,1,2.1666666666666665,13,6,6
5,20230306,moandbear.com.ph,13.906948266666666,0.66666666666666663,9,1.5,1,15,15,10
6,20230305,connectsys.shop,24.650139,0.5,1,2,1,2,2,1
7,20230305,moandbear.com.ph,11.933742800000001,0.6,3,1.6666666666666667,1.2,6,5,3
8,20230304,moandbear.com.ph,421.26791699999995,0.33333333333333331,2,1.5,2.3333333333333335,7,3,2
9,20230304,connectsys.shop,17.537123,0.33333333333333331,0,3,1.6666666666666667,5,3,1


In [8]:
audience_overview_df

,date,hostName,averageSessionDuration,bounceRate,newUsers,sessionsPerUser,screenPageViewsPerSession,screenPageViews,sessions,activeUsers
0,20230310,moandbear.com.ph,7.0485653333333333,1,2,1.5,1.3333333333333333,4,3,2
1,20230309,moandbear.com.ph,46.534410384615384,0.69230769230769229,10,1.0833333333333333,1.4615384615384615,19,13,12
2,20230308,moandbear.com.ph,90.478372153846152,0.30769230769230771,10,1.0833333333333333,2.3846153846153846,31,13,12
3,20230307,moandbear.com.ph,356.8390975,0.33333333333333331,6,1,2.3333333333333335,14,6,6
4,20230306,connectsys.shop,507.05419616666671,0.5,6,1,2.1666666666666665,13,6,6
5,20230306,moandbear.com.ph,13.906948266666666,0.66666666666666663,9,1.5,1,15,15,10
6,20230305,connectsys.shop,24.650139,0.5,1,2,1,2,2,1
7,20230305,moandbear.com.ph,11.933742800000001,0.6,3,1.6666666666666667,1.2,6,5,3
8,20230304,moandbear.com.ph,421.26791699999995,0.33333333333333331,2,1.5,2.3333333333333335,7,3,2
9,20230304,connectsys.shop,17.537123,0.33333333333333331,0,3,1.6666666666666667,5,3,1


In [9]:
query_map["AudienceByHour"] = {
    "metrics": [
        "averageSessionDuration",
        "bounceRate",
        "newUsers",
        "sessionsPerUser",
        "screenPageViewsPerSession",
        "screenPageViews",
        "sessions",
        "activeUsers", # not sure if this is what we want for the api
    ],
    "dimensions": [
        "date",
        "dateHour",
        "hostName",
    ],
    "order_by": "dateHour",
}

In [10]:
audience_by_hour_df = pull_from_ga_into_df(
    query_map["AudienceByHour"]["dimensions"], 
    query_map["AudienceByHour"]["metrics"],
    query_map["AudienceByHour"]["order_by"],
)
audience_by_hour_df

,date,dateHour,hostName,averageSessionDuration,bounceRate,newUsers,sessionsPerUser,screenPageViewsPerSession,screenPageViews,sessions,activeUsers
0,20230310,2023031006,moandbear.com.ph,0,1,1,0,1,1,1,0
1,20230310,2023031005,moandbear.com.ph,21.145696,1,1,1,2,2,1,1
2,20230310,2023031001,moandbear.com.ph,0,1,0,1,1,1,1,1
3,20230309,2023030923,moandbear.com.ph,3.712433,1,2,1,1,2,2,2
4,20230309,2023030918,moandbear.com.ph,0,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...
163,20230222,2023022200,uat.moandbear.com.ph,172.744891,0,1,1,2,2,1,1
164,20230221,2023022122,uat.moandbear.com.ph,1210.965848,1,0,1,3,3,1,1
165,20230221,2023022122,moandbear.com.ph,13.9275805,0.5,1,1,1.5,3,2,2
166,20230221,2023022122,uat.connectsys.shop,5.643172,1,1,1,1,1,1,1


In [11]:
query_map["AudienceByCategory"] = {
    "metrics": [
        "newUsers",
        "activeUsers", # not sure if this is what we want for the api
    ],
    "dimensions": [
        "date",
        "country",
        "deviceCategory",
        "language",
#         "pagePath", ## this makes the request incompatible
    ],
    "order_by": "date",
}

In [12]:
audience_by_category_df = pull_from_ga_into_df(
    query_map["AudienceByCategory"]["dimensions"], 
    query_map["AudienceByCategory"]["metrics"],
    query_map["AudienceByCategory"]["order_by"],
)
audience_by_category_df

,date,country,deviceCategory,language,newUsers,activeUsers
0,20230310,Philippines,mobile,English,1,1
1,20230310,United States,desktop,English,1,0
2,20230310,Philippines,tablet,English,0,1
3,20230309,Philippines,mobile,English,5,6
4,20230309,Philippines,desktop,English,2,2
...,...,...,...,...,...,...
85,20230221,Philippines,desktop,English,4,4
86,20230221,Philippines,mobile,English,2,2
87,20230221,Singapore,desktop,English,1,1
88,20230221,Singapore,mobile,English,1,2


In [16]:
query_map["GaOrders"] = {
    "metrics": [
        "purchaseRevenue",
        "activeUsers", # not sure if this is what we want for the api
    ],
    "dimensions": [
        "month",
        "date",
        "transactionId",
        "campaignName",
    ],
    "order_by": "campaignName",
}

In [17]:
ga_orders_df = pull_from_ga_into_df(
    query_map["GaOrders"]["dimensions"], 
    query_map["GaOrders"]["metrics"],
    query_map["GaOrders"]["order_by"],
)
ga_orders_df

,month,date,transactionId,campaignName,purchaseRevenue,activeUsers
0,03,20230308,384,Mo and Bear Instagram,501.556972,1
1,02,20230228,382,(referral),0,1
2,02,20230228,427,(referral),0,1
3,02,20230224,380,(direct),0,1
4,02,20230225,422,(direct),0,1
5,02,20230228,424,(direct),0,1
6,02,20230228,426,(direct),0,1
7,02,20230228,428,(direct),0,1
8,02,20230228,429,(direct),0,1
9,02,20230228,430,(direct),0,1


In [20]:
query_map["PageTitleViews"] = {
    "metrics": [
        "screenPageViews", # not sure either
    ],
    "dimensions": [
        "date",
        "hostName",
        "landingPagePlusQueryString",
        "pagePath", # could alternatively be pageLocation
        "pageTitle",
        "percentScrolled", # assuming this is page depth
    ],
    "order_by": "date",
}

In [21]:
page_title_views_df = pull_from_ga_into_df(
    query_map["PageTitleViews"]["dimensions"], 
    query_map["PageTitleViews"]["metrics"],
    query_map["PageTitleViews"]["order_by"],
)
page_title_views_df

,date,hostName,landingPagePlusQueryString,pagePath,pageTitle,percentScrolled,screenPageViews
0,20230310,moandbear.com.ph,/,/,Shopping Cart Software & Ecommerce Software So...,,2
1,20230310,moandbear.com.ph,/?fbclid=PAAabCu8MfAQKGlAOY-1GGeC-F_izq94OVBYU...,/,Shopping Cart Software & Ecommerce Software So...,,1
2,20230310,moandbear.com.ph,/pet-care/,/pet-care/,Pet Care,,1
3,20230309,moandbear.com.ph,/,/,Shopping Cart Software & Ecommerce Software So...,,8
4,20230309,moandbear.com.ph,/brands/adidas-accessories-kids-fk3484-bp-powe...,/brands/adidas-accessories-kids-fk3484-bp-powe...,Page Not Found,,1
...,...,...,...,...,...,...,...
329,20230221,moandbear.com.ph,/pet-care/pet-food/?fbclid=IwAR3qyITVwt2bp3p6k...,/pet-care/pet-food/,Pet Care :: Pet Food - Mo&Bear Categories,,1
330,20230221,uat.connectsys.shop,/vendor,/vendor,Page Not Found,,1
331,20230221,uat.moandbear.com.ph,/brands/happy-life/,/,Shopping Cart Software & Ecommerce Software So...,,1
332,20230221,uat.moandbear.com.ph,/brands/happy-life/,/brands/happy-life/,Happy Life,,1


In [39]:
query_map["Acquisition"] = {
    "metrics": [
        "averageSessionDuration",
        "bounceRate",
        "totalRevenue",
        "screenPageViewsPerSession",
        "sessions",
        "newUsers",
        "totalUsers",
    ],
    "dimensions": [
        "date",
#         "campaignName",
        "month",
        "transactionId",
#         "sourceMedium",
        "adSourceName", # alternatively firstUserGoogleAdsAdNetworkType, 
        # googleAdsAdNetworkType, sessionGoogleAdsAdNetworkType
        "hostName",
    ],
    "order_by": "date",
}

In [40]:
acquisition_df = pull_from_ga_into_df(
    query_map["Acquisition"]["dimensions"], 
    query_map["Acquisition"]["metrics"],
    query_map["Acquisition"]["order_by"],
)
acquisition_df

,date,month,transactionId,adSourceName,hostName,averageSessionDuration,bounceRate,totalRevenue,screenPageViewsPerSession,sessions,newUsers,totalUsers
0,20230310,03,,,moandbear.com.ph,7.0485653333333333,1,0,1.3333333333333333,3,2,3
1,20230309,03,(not set),,moandbear.com.ph,46.534410384615384,0.69230769230769229,0,1.4615384615384615,13,10,12
2,20230308,03,(not set),,moandbear.com.ph,90.476962384615391,0.30769230769230771,0,2.3846153846153846,13,10,13
3,20230308,03,384,,moandbear.com.ph,0.018327,1,501.557137,0,1,0,1
4,20230307,03,(not set),,moandbear.com.ph,356.8390975,0.33333333333333331,0,2.3333333333333335,6,6,6
5,20230306,03,(not set),,connectsys.shop,507.05419616666671,0.5,0,2.1666666666666665,6,6,6
6,20230306,03,(not set),,moandbear.com.ph,13.906948266666666,0.66666666666666663,0,1,15,9,12
7,20230305,03,(not set),,connectsys.shop,24.650139,0.5,0,1,2,1,2
8,20230305,03,(not set),,moandbear.com.ph,11.933742800000001,0.6,0,1.2,5,3,4
9,20230304,03,(not set),,moandbear.com.ph,421.26791699999995,0.33333333333333331,0,2.3333333333333335,3,2,2
